In [1]:
# tf tools
import tensorflow as tf

# image processsing
from tensorflow.keras.preprocessing.image import (load_img,
                                                  img_to_array,
                                                  ImageDataGenerator)
# VGG16 model
from tensorflow.keras.applications.vgg16 import (preprocess_input,
                                                 decode_predictions,
                                                 VGG16)

# layers
from tensorflow.keras.layers import (Flatten, 
                                     Dense, 
                                     Dropout, 
                                     BatchNormalization)
# generic model object
from tensorflow.keras.models import Model

# optimizers
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import SGD

#scikit-learn
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

# for plotting
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

2023-05-16 17:36:43.801329: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-16 17:36:43.862236: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-16 17:36:43.868303: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-16 17:36:44.803698: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
#loading VGG16 without the classification layers and disableling the convolutional layers
model = VGG16()

553467096/553467096 [==============================] - 15s 0us/step


In [3]:
def lables(type):
    df = pd.read_json(f"../../images/metadata/{type}_data.json", lines=True)
    lables = df['class_label']
    unique_classes = lables.unique()
    
    return lables, unique_classes

In [4]:
train_lables = lables("train")[0]
test_lables = lables("test")[0]

In [5]:
# integers to one-hot vectors
lb = LabelBinarizer()
train_lables = lb.fit_transform(train_lables)
test_lables = lb.fit_transform(test_lables)

labelNames = lables("train")[1]
labelNames = labelNames.tolist()

In [6]:
train_data_dir = os.path.join("../../images/")
test_data_dir = os.path.join("../../images/")
batch_size = 32
image_size = (224, 224)  # Adjust the dimensions as per your requirements

In [7]:
# load model without classifier layers
model = VGG16(include_top=False, 
              pooling='avg',
              input_shape=(32, 32, 3))

# mark loaded layers as not trainable
for layer in model.layers:
    layer.trainable = False

58889256/58889256 [==============================] - 2s 0us/step


In [8]:
tf.keras.backend.clear_session()
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(128, activation='relu')(flat1)
output = Dense(10, activation='softmax')(class1)

# define new model
model = Model(inputs=model.inputs, 
              outputs=output)
# summarize
#model.summary()

In [9]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)
sgd = SGD(learning_rate=lr_schedule)

model.compile(optimizer=sgd,
              loss='sparse_categorical_crossentropy', #from https://www.kaggle.com/general/197993
              metrics=['accuracy'])

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(32, 32),
    batch_size=batch_size,
    class_mode='binary',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(32, 32),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation') # set as validation data

H = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 10) 

Found 84933 images belonging to 4 classes.
Found 21233 images belonging to 4 classes.


/tmp/ipykernel_867/210005581.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  H = model.fit_generator(


Epoch 1/10


2023-05-16 17:38:07.886826: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


 682/2654 [======>.......................] - ETA: 1:40:16 - loss: 0.5325 - accuracy: 0.8567

KeyboardInterrupt: 

In [12]:
def plot_history(H, epochs):
    plt.style.use("seaborn-colorblind")

    plt.figure(figsize=(12,6))
    plt.subplot(1,2,1)
    plt.plot(np.arange(0, epochs), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, epochs), H.history["val_loss"], label="val_loss", linestyle=":")
    plt.title("Loss curve")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.tight_layout()
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(np.arange(0, epochs), H.history["accuracy"], label="train_acc")
    plt.plot(np.arange(0, epochs), H.history["val_accuracy"], label="val_acc", linestyle=":")
    plt.title("Accuracy curve")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.tight_layout()
    plt.legend()
    plt.show()

In [13]:
plot_history(H, 1)

NameError: name 'H' is not defined

In [ ]:
predictions = model.predict(test_data_dir, batch_size=128)
classification_report(train_data_dir.argmax(axis=1),
                            predictions.argmax(axis=1),
                            target_names=labelNames)

#saving report to text file
report_path = "../out/classification_report.txt"

text_file = open(report_path, "w")
n = text_file.write(report)
text_file.close()